<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Named Entity Recognition
  </div> 
  
<div style="
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
     Finetuning on CHIA (PubMedBert)
  </div> 


  <div style="
      font-size: 15px; 
      line-height: 12px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Jean-baptiste AUJOGUE
  </div> 


  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  December 2022
  </div>

<a id="TOC"></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import re
import random
import copy
import string

# data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import (
    Dataset, 
    DatasetDict,
    ClassLabel, 
    Features, 
    Sequence, 
    Value,
)
from transformers import DataCollatorForTokenClassification

# DL
import torch
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    AutoModelForTokenClassification, 
    TrainingArguments, 
    Trainer,
    pipeline,
    set_seed,
)
import evaluate

# viz
from IPython.display import HTML

C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Transformers settings

In [3]:
transformers.__version__

'4.22.2'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

#### Custom paths & imports

In [5]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_data = os.path.join(path_to_repo, 'datasets', 'chia', 'chia-ner')
path_to_logs = os.path.join(path_to_repo, 'logs', 'NER')
path_to_save_mlm = os.path.join(path_to_repo, 'saves', 'MLM')
path_to_save_ner = os.path.join(path_to_repo, 'saves', 'NER')
path_to_src  = os.path.join(path_to_repo, 'src')

In [6]:
sys.path.insert(0, path_to_src)

In [7]:
from nlptools.ner.preprocessing import tokenize_and_align_categories, create_labels
from nlptools.ner.metrics import compute_metrics, compute_metrics_finegrained
from nlptools.ner.postprocessing import parse_trf_ner_output, remove_entity_overlaps, correct_entity_boundaries
from nlptools.ner.visualization import render_ner_as_html

#### Constants

In [8]:
run_name = 'notebook1'
dataset_name = 'chia'
hub_model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
base_model_name = 'pubmedbert'
final_model_name = '{}-{}'.format(base_model_name, dataset_name)

<a id="data"></a>

# 1. Dataset

[Table of content](#TOC)


## 1.1 Load BIO corpus

[Table of content](#TOC)

In [9]:
df_bio = pd.read_csv(os.path.join(path_to_data, 'chia_bio.tsv'), sep = "\t")

df_bio.head(10)

,Id,Sequence_id,Mention,Category
0,NCT00050349_exc,NCT00050349_exc_0,Patients,O
1,NCT00050349_exc,NCT00050349_exc_0,with,O
2,NCT00050349_exc,NCT00050349_exc_0,symptomatic,B-Qualifier
3,NCT00050349_exc,NCT00050349_exc_0,,O
4,NCT00050349_exc,NCT00050349_exc_0,CNS,B-Condition
5,NCT00050349_exc,NCT00050349_exc_0,metastases,I-Condition
6,NCT00050349_exc,NCT00050349_exc_0,,O
7,NCT00050349_exc,NCT00050349_exc_0,or,O
8,NCT00050349_exc,NCT00050349_exc_0,leptomeningeal,B-Condition
9,NCT00050349_exc,NCT00050349_exc_0,involvement,I-Condition


In [10]:
class_labels = sorted(list(set(df_bio.Category.unique())))
class_labels = ClassLabel(names = class_labels)

In [11]:
label2id = class_labels._str2int
id2label = {i: l for l, i in label2id.items()}

## 1.2 Generate train / valid / test dataset

[Table of content](#TOC)

In [12]:
def get_item_list(df, grp_col, item_col):
    return df.groupby(grp_col).apply(lambda g: g[item_col].tolist()).tolist()



def convert_dataframe_to_dataset(df_bio):
    data = {
        'ids': df_bio.Sequence_id.unique().tolist(),
        'mentions': get_item_list(df_bio, 'Sequence_id', 'Mention'),
        'categories': get_item_list(df_bio, 'Sequence_id', 'Category'),
    }
    return data

In [13]:
# dataset separation: 800 trials for training, 100 trials for validation and 100 trials for testing
ids_bio = sorted(list(set(df_bio.Id.apply(lambda i: i.split('_')[0]))))

ids_trn, ids_dev = train_test_split(ids_bio, train_size = 0.8, random_state = 13, shuffle = True)
ids_dev, ids_tst = train_test_split(ids_dev, train_size = 0.5, random_state = 13, shuffle = True)

len(ids_bio), len(ids_trn), len(ids_dev), len(ids_tst)

(1000, 800, 100, 100)

In [14]:
df_trn = df_bio[df_bio.Id.apply(lambda i: i.split('_')[0]).isin(ids_trn)]
df_dev = df_bio[df_bio.Id.apply(lambda i: i.split('_')[0]).isin(ids_dev)]
df_tst = df_bio[df_bio.Id.apply(lambda i: i.split('_')[0]).isin(ids_tst)]

In [15]:
dict_bio = convert_dataframe_to_dataset(df_bio)
dict_trn = convert_dataframe_to_dataset(df_trn)
dict_dev = convert_dataframe_to_dataset(df_dev)
dict_tst = convert_dataframe_to_dataset(df_tst)

In [16]:
# 'ner_tags': Sequence(class_labels),
features = Features({
    'ids': Value(dtype = 'string'), 
    'mentions': Sequence(Value(dtype = 'string')), 
    'categories': Sequence(Value(dtype = 'string')),
})

In [17]:
raw_datasets = DatasetDict({
    'trn': Dataset.from_dict(dict_trn, features = features),
    'dev': Dataset.from_dict(dict_dev, features = features),
    'tst': Dataset.from_dict(dict_tst, features = features),
    'all': Dataset.from_dict(dict_bio, features = features),
})

## 1.3 Apply model-specific tokenization

[Table of content](#TOC)


In [18]:
tokenizer_path = os.path.join(path_to_save_mlm, base_model_name, 'tokenizer')

try:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    print('Tokenizer loaded from local checkpoint.')
except:
    tokenizer = AutoTokenizer.from_pretrained(hub_model_name)
    tokenizer.save_pretrained(tokenizer_path)
    print('Tokenizer downloaded from Huggingface model hub.')

Tokenizer loaded from local checkpoint.


In [19]:
B_I_mapping = {l: 'I'+l[1:] for l in class_labels.names if l.startswith('B-')}
B_I_mapping

{'B-Condition': 'I-Condition',
 'B-Device': 'I-Device',
 'B-Drug': 'I-Drug',
 'B-Measurement': 'I-Measurement',
 'B-Observation': 'I-Observation',
 'B-Person': 'I-Person',
 'B-Procedure': 'I-Procedure',
 'B-Qualifier': 'I-Qualifier',
 'B-Temporal': 'I-Temporal',
 'B-Value': 'I-Value'}

In [20]:
tokenized_datasets = raw_datasets.map(
    function = lambda examples: tokenize_and_align_categories(tokenizer, examples, B_I_mapping), 
    batched  = True,
)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  6.08ba/s]


<AxesSubplot: ylabel='Frequency'>

In [21]:
# distribution of lengths
lengths = [len(ts) for ts in tokenized_datasets['all']['input_ids']]
lengths = [l for l in lengths if l<= 100]

pd.DataFrame(lengths, columns = ['Length']).plot.hist(bins = 100)

In [22]:
for c in ['ids', 'mentions', 'categories', 'tokens', 'token_categories', 'input_ids']:
    print(c, ':', tokenized_datasets['tst'][c][0])

ids : NCT00122070_exc_0
mentions : ['Are', 'pregnant', ' ', 'or', 'lactating', '.']
categories : ['O', 'B-Condition', 'O', 'O', 'B-Condition', 'O']
tokens : ['[CLS]', 'are', 'pregnant', 'or', 'lactating', '.', '[SEP]']
token_categories : [None, 'O', 'B-Condition', 'O', 'B-Condition', 'O', None]
input_ids : [2, 2032, 7808, 2014, 25984, 18, 3]


## 1.4 Map categories to integers

[Table of content](#TOC)


In [23]:
tokenized_datasets = tokenized_datasets.map(
    function = lambda examples: create_labels(examples, class_labels), 
    batched  = True,
)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  3.63ba/s]


In [24]:
for c in ['ids', 'mentions','categories', 'tokens', 'token_categories', 'input_ids', 'labels']:
    print(c, ':', tokenized_datasets['tst'][c][0])

ids : NCT00122070_exc_0
mentions : ['Are', 'pregnant', ' ', 'or', 'lactating', '.']
categories : ['O', 'B-Condition', 'O', 'O', 'B-Condition', 'O']
tokens : ['[CLS]', 'are', 'pregnant', 'or', 'lactating', '.', '[SEP]']
token_categories : [None, 'O', 'B-Condition', 'O', 'B-Condition', 'O', None]
input_ids : [2, 2032, 7808, 2014, 25984, 18, 3]
labels : [-100, 20, 0, 20, 0, 20, -100]


In [25]:
max([len(c) for c in tokenized_datasets['trn']['input_ids']])

447

<a id="albert"></a>

# 2. Model finetuning

[Table of content](#TOC)

## 2.1 BERT model for Token Classification

[Table of content](#TOC)


In [26]:
# make model creation & training deterministic
set_seed(42)

In [27]:
base_model_path = os.path.join(path_to_save_mlm, base_model_name, 'model')

try:
    model = AutoModelForTokenClassification.from_pretrained(base_model_path, label2id = label2id, id2label = id2label)
    print('Model loaded from local checkpoint.')
except:
    model = AutoModelForMaskedLM.from_pretrained(hub_model_name)
    model.save_pretrained(base_model_path)
    model = AutoModelForTokenClassification.from_pretrained(base_model_path, label2id = label2id, id2label = id2label)
    print('Model downloaded from Huggingface model hub.')

Some weights of the model checkpoint at C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\MLM\pubmedbert\model were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

Model loaded from local checkpoint.


In [28]:
model.num_parameters()

108907797

In [29]:
model.config

BertConfig {
  "_name_or_path": "C:\\Users\\jb\\Desktop\\NLP\\perso - Transformers for NLP\\saves\\MLM\\pubmedbert\\model",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-Condition",
    "1": "B-Device",
    "2": "B-Drug",
    "3": "B-Measurement",
    "4": "B-Observation",
    "5": "B-Person",
    "6": "B-Procedure",
    "7": "B-Qualifier",
    "8": "B-Temporal",
    "9": "B-Value",
    "10": "I-Condition",
    "11": "I-Device",
    "12": "I-Drug",
    "13": "I-Measurement",
    "14": "I-Observation",
    "15": "I-Person",
    "16": "I-Procedure",
    "17": "I-Qualifier",
    "18": "I-Temporal",
    "19": "I-Value",
    "20": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-Condition": 0,
    "B-Device": 1,
    "B-Drug": 2,
    "B-Measurement": 3,
    "B-Observation": 4,
  

## 2.2 Compute finetuning metrics

[Table of content](#TOC)

In [ ]:
batch_size = 6

In [ ]:
model = model.to(device)
model = model.train()

In [ ]:
args = TrainingArguments(
    # training args
    learning_rate = 2e-5,
    weight_decay = 1e-4,
    num_train_epochs = 4,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,

    # logging args
    output_dir = os.path.join(path_to_save_ner, '_checkpoints'),
    logging_dir = os.path.join(path_to_logs, final_model_name, run_name),
    evaluation_strategy = 'steps',
    save_strategy = 'no',
    logging_steps = 500,
    report_to = ['tensorboard'],
    log_level = 'warning',
)

In [ ]:
metric = evaluate.load("seqeval")

In [ ]:
trainer = Trainer(
    model,
    args,
    data_collator = DataCollatorForTokenClassification(tokenizer),
    train_dataset = tokenized_datasets['trn'],
    eval_dataset  = tokenized_datasets['dev'],
    compute_metrics = lambda p: compute_metrics_finegrained(p, metric, class_labels.names),
)

In [ ]:
trainer.train()

In [ ]:
test_results = trainer.evaluate(eval_dataset = tokenized_datasets['tst'], metric_key_prefix = 'test')

In [ ]:
for k, v in test_results.items():
    print(str(k) + ' ' + '-'*(30 - len(k)) + ' {:2f}'.format(100*v))

In [ ]:
del model

torch.cuda.empty_cache()

## 2.3 Finetune final model

[Table of content](#TOC)

In [30]:
# make model creation & training deterministic
set_seed(42)

In [31]:
base_model_path = os.path.join(path_to_save_mlm, base_model_name, 'model')

model = AutoModelForTokenClassification.from_pretrained(base_model_path, label2id = label2id, id2label = id2label)

Some weights of the model checkpoint at C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\MLM\pubmedbert\model were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

In [32]:
batch_size = 6

In [33]:
model = model.to(device)
model = model.train()

In [34]:
args = TrainingArguments(
    # training args
    learning_rate = 2e-5,
    weight_decay = 1e-4,
    num_train_epochs = 4,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,

    # logging args
    output_dir = os.path.join(path_to_save_ner, '_checkpoints'),
    evaluation_strategy = 'no',
    save_strategy = 'no',
)

In [35]:
metric = evaluate.load("seqeval")

In [36]:
trainer = Trainer(
    model,
    args,
    data_collator = DataCollatorForTokenClassification(tokenizer),
    train_dataset = tokenized_datasets['all'],
)

In [37]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: token_categories, mentions, categories, ids, tokens. If token_categories, mentions, categories, ids, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12633
  Num Epochs = 4
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 8424
You're using a BertTokenizerFast tokenizer. Please note that with a fast t

Step,Training Loss
500,1.080700
1000,0.703700
1500,0.638700
2000,0.634100
2500,0.550000
3000,0.507800
3500,0.485400
4000,0.495800
4500,0.445100
5000,0.404100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8424, training_loss=0.49616543406661306, metrics={'train_runtime': 945.6323, 'train_samples_per_second': 53.437, 'train_steps_per_second': 8.908, 'total_flos': 1033997184631044.0, 'train_loss': 0.49616543406661306, 'epoch': 4.0})

In [38]:
model = model.to('cpu')

In [39]:
tokenizer.save_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'tokenizer'))
model.save_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'model'))

tokenizer config file saved in C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\NER\pubmedbert-chia\notebook1\tokenizer\tokenizer_config.json
Special tokens file saved in C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\NER\pubmedbert-chia\notebook1\tokenizer\special_tokens_map.json
Configuration saved in C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\NER\pubmedbert-chia\notebook1\model\config.json
Model weights saved in C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\NER\pubmedbert-chia\notebook1\model\pytorch_model.bin


<a id="inference"></a>

# 3. Inference

[Table of content](#TOC)

In [40]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'tokenizer'))
model = AutoModelForTokenClassification.from_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'model'))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\NER\pubmedbert-chia\notebook1\model\config.json
Model config BertConfig {
  "_name_or_path": "C:\\Users\\jb\\Desktop\\NLP\\perso - Transformers for NLP\\saves\\NER\\pubmedbert-chia\\notebook1\\model",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-Condition",
    "1": "B-Device",
    "2": "B-Drug",
    "3": "B-Measurement",
    "4": "B-Observation",
    "5": "B-Person",
    "6": "B-Procedure",
    "7": "B-Qualifier",
    "8": "B-Temporal",
    "9": "B-Value",
    "10": "I-Condition",
    "11": "I-Device",
    "12": "I-Drug",
    "13": "I-Measurement",
    "14": "I-Obse

In [41]:
ner = pipeline(
    task = 'ner', 
    model = model, 
    tokenizer = tokenizer,
    framework = 'pt',
    aggregation_strategy = 'simple',
)

In [42]:
sent = 'Radiologic examination of chest'


df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [43]:
sent = 'Participants with at least two of the following: serum M-protein >= 0.5 g/dL, urine M-protein >=200 mg/24 hours, an involved FLC assay >=100 mg/L, and a serum FLC ratio <0.26 or a serum FLC ratio  >1.65'


df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [44]:
sent = 'Participants with at least two of the following: Serum M-protein >= 0.5 g/dL measured using serum protein immunoelectrophoresis, Urine M-protein >=200 mg/24 hours measured using urine protein immunoelectrophoresis, Serum free light chain (FLC) assay with an involved FLC assay >=100 mg/L, and a serum FLC ratio < 0.26 or a serum FLC ratio  > 1.65'


df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [45]:
sent = 'Polyneuropathy of other causes, including but not limited to hereditary demyelinating neuropathies, neuropathies secondary to infection or systemic disease, diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor neuropathy, monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory CIDP and acquired demyelinating symmetric (DADS) neuropathy (also known as distal CIDP).'

df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [46]:
sent = 'Adults and adolescent patients with a physician diagnosis of asthma ' +\
    'for ≥12 months, based on the Global Initiative for Asthma (GINA) 2014 Guidelines ' +\
    'and the followingcriteria: A) Existing treatment with medium to high dose ICS ' +\
    '(≥250 mcg of fluticasone propionate twice daily or equipotent ICS daily dosage ' +\
    'to a maximum of 2000 mcg/day of fluticasone propionate or equivalent) in combination ' +\
    'with a second controller (eg, LABA, LTRA) for at least 3 months with a stable dose ≥1 ' +\
    'month prior to Visit 1.'

df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [47]:
sent = 'Biologics treatment: Cell-depleting agents, eg. Rituximab Drug within 6 months before baseline or until lymphocyte count returns to normal, Other biologics: within 5 half-lives or 16 weeks prior baseline.'

df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [48]:
sents = '''Active hepatitis or patients with positive HBsAg, or patients with positive HBcAb plus positive HBV DNA, or positive HCV antibody (confirmed with presence of HCV RNA if needed) at screening.

History of alcohol or drug abuse within 2 years of the screening visit.

History of HIV infection or positive HIV serology at screening.

History of malignancy within 5 years before screening, except completely treated cervix carcinoma, completely treated and resolved non-metastatic squamous or basal cell carcinoma of the skin.

Infection requires systemic antibiotics, antivirals, antiparasitics, antiprotozoals, antifungals treatment within 2 weeks before baseline, superficial skin infections within 1 week before baseline visit.

Initiation AD treatment with prescription moisturizers or moisturizers containing  additives such as ceramide, hyaluronic acid, urea, or filaggrin degradation products during the screening period.

Known or suspected history of immunosuppression, including history of invasive opportunistic infections, despite infection resolution, or unusually frequent, recurrent, or prolonged infections.
'''
df_list = []
html = ''
for sent in sents.split('\n'):
    if sent.strip():
        ents = parse_trf_ner_output(ner(sent))
        ents = correct_entity_boundaries(sent, ents)
        ents = remove_entity_overlaps(ents)
        df_list.append(ents)
        html += render_ner_as_html(sent, ents)

df_ents = pd.concat(df_list, ignore_index = True)
HTML(html)

[Table of content](#TOC)